In [94]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from tensorflow.keras.utils import to_categorical
from cv2 import imread, resize

# Класифікація даних

In [95]:
df = pd.read_csv('fake_job_postings.csv')
df.columns #names of columns

cat_columns = df.columns[df.dtypes == object]
df[cat_columns] = df[cat_columns].fillna(0)

model = Sequential()
model.add(layers.Dense(12, input_dim=8, activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [96]:
X = df.drop("fraudulent",axis=1)
y = df.fraudulent
y = y.astype('int')
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [98]:
model = KerasClassifier(build_fn =model, epochs=150, batch_size=10, verbose=0)
model.fit(X_train, y_train)
model.score(X_test, y_test)


0.9809698563536775


c:\python 3.7\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  """Entry point for launching an IPython kernel.


# Класифікація зображень

In [99]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
%matplotlib inline
import random
from PIL import Image
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import preprocessing
from tensorflow.keras import callbacks
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers

In [100]:
data_path = "Data/Data/"

In [101]:
random.seed(123)
seed = random.randint(0,50)

batch_size = 32
width = 224
height = 224

train_dataset = preprocessing.image_dataset_from_directory(
    directory=data_path,
    validation_split=0.2,
    subset="training",
    seed=seed,
    image_size=(height,width),
    batch_size=batch_size
)

Found 9787 files belonging to 15 classes.
Using 7830 files for training.


In [102]:
val_dataset = preprocessing.image_dataset_from_directory(
    directory=data_path,
    validation_split=0.2,
    subset="validation",
    seed=seed,
    image_size=(height, width),
    batch_size=batch_size
)

Found 9787 files belonging to 15 classes.
Using 1957 files for validation.


In [103]:
data_augmentation = Sequential(layers=[
                                tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical", input_shape=(height,width,3)),
                                tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
                                tf.keras.layers.experimental.preprocessing.RandomZoom(0.1), 
                    ], name="data_augmentation")

In [104]:
model = Sequential()
base_model = tf.keras.applications.MobileNet(input_shape=(width,height,3), weights="imagenet",include_top=False)

model.add(data_augmentation)
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())  
model.add(layers.BatchNormalization())
    
model.add(layers.Dense(units=1024, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(units=1024, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(units=512, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(classes_count, activation='sigmoid',name='final_output'))


model.compile(optimizer=optimizers.Adam(learning_rate=1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [105]:
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5,
    verbose=1, 
)

Epoch 1/5
245/245 [==============================] - 1158s 5s/step - loss: 1.6826 - accuracy: 0.4843 - val_loss: 2.3092 - val_accuracy: 0.3306
Epoch 2/5
245/245 [==============================] - 1102s 4s/step - loss: 1.0637 - accuracy: 0.6898 - val_loss: 2.7043 - val_accuracy: 0.4456
Epoch 3/5
245/245 [==============================] - 1092s 4s/step - loss: 0.8886 - accuracy: 0.7465 - val_loss: 1.1589 - val_accuracy: 0.6857
Epoch 4/5
245/245 [==============================] - 1088s 4s/step - loss: 0.7547 - accuracy: 0.7840 - val_loss: 2.0207 - val_accuracy: 0.5192
Epoch 5/5
245/245 [==============================] - 1073s 4s/step - loss: 0.6710 - accuracy: 0.8153 - val_loss: 1.3323 - val_accuracy: 0.6653


In [106]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 data_augmentation (Sequenti  (None, 224, 224, 3)      0         
 al)                                                             
                                                                 
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d_2   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 batch_normalization_2 (Batc  (None, 1024)             4096      
 hNormalization)                                                 
                                                                 
 dense_35 (Dense)            (None, 1024)            

# Прогнозування значень часового ряду

In [87]:
from sklearn.preprocessing import MinMaxScaler
data_test = pd.read_csv('DailyDelhiClimateTest.csv')
data_train = pd.read_csv('DailyDelhiClimateTest.csv')
temp_train = data_train.iloc[:,1:2]
temp_test = data_test.iloc[:,1:2]
ss= MinMaxScaler(feature_range=(0,1))
temp_train= ss.fit_transform(temp_train)
temp_test= ss.fit_transform(temp_test)

In [91]:
def create_dataset(dataset, look_back=1):
    X_data, Y_data = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        X_data.append(a)
        Y_data.append(dataset[i + look_back, 0])
    return np.array(X_data), np.array(Y_data)

look_back = 1
X_train, Y_train = create_dataset(temp_train, look_back)
X_test, Y_test = create_dataset(temp_test, look_back)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))


In [92]:
from keras.layers.core import Dense, Dropout
model_temp = Sequential()
model_temp.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model_temp.add(Dropout(0.2))
model_temp.add(LSTM(units = 100, return_sequences = True))
model_temp.add(Dropout(0.2))
model_temp.add(LSTM(units = 100, return_sequences = True))
model_temp.add(Dropout(0.2))
model_temp.add(LSTM(units = 50))
model_temp.add(Dropout(0.2))
model_temp.add(Dense(units = 1))
model_temp.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['accuracy'])

model_temp.fit(X_train, Y_train, epochs = 150, batch_size = 32)

Epoch 1/150
4/4 [==============================] - 5s 6ms/step - loss: 0.2737 - accuracy: 0.0089 
Epoch 2/150
4/4 [==============================] - 0s 6ms/step - loss: 0.2586 - accuracy: 0.0089
Epoch 3/150
4/4 [==============================] - 0s 7ms/step - loss: 0.2405 - accuracy: 0.0089
Epoch 4/150
4/4 [==============================] - 0s 8ms/step - loss: 0.2202 - accuracy: 0.0089
Epoch 5/150
4/4 [==============================] - 0s 8ms/step - loss: 0.1912 - accuracy: 0.0089
Epoch 6/150
4/4 [==============================] - 0s 8ms/step - loss: 0.1544 - accuracy: 0.0089
Epoch 7/150
4/4 [==============================] - 0s 8ms/step - loss: 0.1131 - accuracy: 0.0089
Epoch 8/150
4/4 [==============================] - 0s 7ms/step - loss: 0.0687 - accuracy: 0.0179
Epoch 9/150
4/4 [==============================] - 0s 7ms/step - loss: 0.0436 - accuracy: 0.0179
Epoch 10/150
4/4 [==============================] - 0s 7ms/step - loss: 0.0602 - accuracy: 0.0179
Epoch 11/150
4/4 [==========

4/4 [==============================] - 0s 8ms/step - loss: 0.0075 - accuracy: 0.0179
Epoch 85/150
4/4 [==============================] - 0s 8ms/step - loss: 0.0076 - accuracy: 0.0179
Epoch 86/150
4/4 [==============================] - 0s 8ms/step - loss: 0.0082 - accuracy: 0.0179
Epoch 87/150
4/4 [==============================] - 0s 8ms/step - loss: 0.0088 - accuracy: 0.0179
Epoch 88/150
4/4 [==============================] - 0s 8ms/step - loss: 0.0085 - accuracy: 0.0179
Epoch 89/150
4/4 [==============================] - 0s 7ms/step - loss: 0.0082 - accuracy: 0.0179
Epoch 90/150
4/4 [==============================] - 0s 7ms/step - loss: 0.0083 - accuracy: 0.0179
Epoch 91/150
4/4 [==============================] - 0s 7ms/step - loss: 0.0068 - accuracy: 0.0179
Epoch 92/150
4/4 [==============================] - 0s 7ms/step - loss: 0.0079 - accuracy: 0.0179
Epoch 93/150
4/4 [==============================] - 0s 7ms/step - loss: 0.0076 - accuracy: 0.0179
Epoch 94/150
4/4 [===============